In [1]:
#原始寫法(研究中使用)
import pandas as pd
import urllib.request, json

with urllib.request.urlopen("http://iflastandards.info/ns/fr/frbr/frbrer.jsonld") as url:
    info_ontology_FRBR = json.loads(url.read().decode())

ontology_in_table = pd.DataFrame(info_ontology_FRBR)

property_domain = []
property_range = []
property_label = []
property_content = []
property_definition = []

for ontology_property in range(11,217):
    property_domain += [ontology_in_table["@graph"][ontology_property]["domain"]["label"]]
    #property_range += [ontology_in_table["@graph"][ontology_property]["range"]["label"]]
    property_label += [ontology_in_table["@graph"][ontology_property]["@id"].replace("http://iflastandards.info/ns/fr/frbr/frbrer/","")]
    property_content += [ontology_in_table["@graph"][ontology_property]["label"]["en"]]
    property_definition += [ontology_in_table["@graph"][ontology_property]["description"]["en"]]

property_list = pd.DataFrame({"domain": property_domain,
                              "label": property_label,
                              "content": property_content,
                              "defination": property_definition},
                              columns=["label",
                                       "domain",
                                       "content",
                                       "defination"]).set_index("label")

property_list#.to_csv("FRBR屬性撈.csv")

,domain,content,defination
label,,,
P2001,Work,is realized through,[Relates a work to an expression that is the r...
P2002,Expression,is realization of,[Relates an expression to the work that is rea...
P2003,Expression,is embodied in,[Relates an expression to a manifestation that...
P2004,Manifestation,is embodiment of,[Relates a manifestation to an expression that...
P2005,Manifestation,is exemplified by,[Relates a manifestation to an item that is a ...
...,...,...,...
P3097,Expression,has scale (cartographic image or object),[Relates an expression that is cartographic to...
P3098,Expression,has projection (cartographic image or object),[Relates an expression that is cartographic to...
P3099,Expression,has presentation technique (cartographic image...,[Relates an expression that is cartographic to...


In [79]:
#RDFlib寫法
from rdfpandas.graph import to_dataframe
import pandas as pd
import rdflib

g = rdflib.Graph()
g.parse('http://iflastandards.info/ns/fr/frbr/frbrer.nt', format = 'ttl')
df = pd.DataFrame(to_dataframe(g))

entities_dict = {"C1001": "Work",
            "C1002": "Expression",
            "C1003": "Manifestation",
            "C1004": "Item",
            "C1005": "Person",
            "C1006": "Corporate Body",
            "C1007": "Concept",
            "C1008": "Object",
            "C1009": "Event",
            "C1010": "Place"}



df = df.set_index(df.index.str.replace("http://iflastandards.info/ns/fr/frbr/frbrer/", ""))

df = df.replace({"http://metadataregistry.org/uri/RegStatus/": "",
                               "http://iflastandards.info/ns/fr/frbr/frbrer/": "",
                               "http://www.w3.org/2002/07/owl#": "",
                               "rdf:": ""}, regex=True)

#

df = df.reindex(columns=["rdf:type{URIRef}", "rdfs:domain{URIRef}", "rdfs:label{Literal}@en", "rdfs:range{URIRef}"]).iloc[:-1]



df.rename(columns={"rdf:type{URIRef}": 'type', 
                                     'rdfs:domain{URIRef}': 'domain',
                                     "rdfs:label{Literal}@en": "content",
                                     "rdfs:range{URIRef}": "range"}, inplace=True)

df.drop(df.index[0], inplace=True)


df["range"].replace(entities_dict, inplace=True)

df["domain"].replace(entities_dict, inplace=True)

C:\Users\jerry\AppData\Local\Temp/ipykernel_20516/367080289.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.set_index(df.index.str.replace("http://iflastandards.info/ns/fr/frbr/frbrer/", ""))


In [80]:
df#.to_csv("FRBR屬性撈.csv")

,type,domain,content,range
C1001,Class,NaN,Work,NaN
C1002,Class,NaN,Expression,NaN
C1003,Class,NaN,Manifestation,NaN
C1004,Class,NaN,Item,NaN
C1005,Class,NaN,Person,NaN
...,...,...,...,...
P3097,Property,Expression,has scale (cartographic image or object),NaN
P3098,Property,Expression,has projection (cartographic image or object),NaN
P3099,Property,Expression,has presentation technique (cartographic image...,NaN
P3100,Property,Expression,has representation of relief (cartographic ima...,NaN
